# Repo Instrument Example

In this notebook, we will book two different repo instruments into a portfolio, and then continue to roll the principal and interest rate of the two bonds indepenedently. 

## Setup

In [1]:
# Import generic non-LUSID packages
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json
import pytz
import time
from IPython.core.display import HTML

# Import key modules from the LUSID package
import lusid as lu
import lusid.models as lm

# Import key functions from Lusid-Python-Tools and other packages
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias
from lusidtools.lpt.lpt import to_date
from lusidjam import RefreshingToken


# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
# display(HTML("<style>.container { width:90% !important; }</style>"))

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate our user and create our API client
api_factory = lu.utilities.ApiClientFactory(
    token=RefreshingToken(), api_secrets_filename=secrets_path
)

print("LUSID Environment Initialised")
print(
    "LUSID API Version :",
    api_factory.build(lu.api.ApplicationMetadataApi).get_lusid_versions().build_version,
)

LUSID Environment Initialised
LUSID API Version : 0.6.11149.0


In [2]:
portfolio_api = api_factory.build(lu.api.PortfoliosApi)
transaction_portfolios_api = api_factory.build(lu.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lu.api.InstrumentsApi)
quotes_api = api_factory.build(lu.api.QuotesApi)
configuration_recipe_api = api_factory.build(lu.api.ConfigurationRecipeApi)
system_configuration_api = api_factory.build(lu.api.SystemConfigurationApi)
aggregration_api = api_factory.build(lu.api.AggregationApi)

In [3]:
# Define scopes
scope = "ibor"
quotes_scope = "ibor"
portfolio_code = "repoExamplePortfolio"

## 1. Create a Portfolio

We begin by creating a portfolio to store the two repo instruments.

In [4]:
try:
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="USD",
            created="2010-01-01",
            sub_holding_keys=[],
        ),
    )

except lu.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a portfolio with id 'repoExamplePortfolio' because it already exists in scope 'ibor'.


## Create Repo Instruments and Collateral

We will now create two repo instruments and upsert these into Lusid. The two repo instruments are defined as follows:

- Simple repo: no collateral definition
- Repo with collateral: a bond definiton will be provided as collateral

In [5]:
def create_collateral_instrument_definition(
    start_date,
    maturity_date,
    dom_ccy,
    principal,
    coupon_rate,
    currency,
    payment_frequency,
    roll_convention,
    day_count_convention,
    payment_calendars,
    reset_calendars,
    settle_days,
    reset_days
):
    flow_conventions = lm.FlowConventions(
        currency=currency,
        payment_frequency=payment_frequency,
        roll_convention=roll_convention,
        day_count_convention=day_count_convention,
        payment_calendars=payment_calendars,
        reset_calendars=reset_calendars,
        settle_days=settle_days,
        reset_days=reset_days,
    )

    bond = lm.Bond(
        start_date=start_date,
        maturity_date=maturity_date,
        dom_ccy=dom_ccy,
        principal=principal,
        coupon_rate=coupon_rate,
        flow_conventions=flow_conventions,
        identifiers={},
        instrument_type="Bond",
        calculation_type="Standard",
    )
    
    return bond

In [6]:
def create_repo_instrument(
    name,
    identifier,
    start_date,
    maturity_date,
    dom_ccy,
    accrual_basis,
    purchase_price,
    repo_rate,
    collateral
):
    repo_instrument = lm.Repo(
        instrument_type="Repo",
        start_date=start_date,
        maturity_date=maturity_date,
        dom_ccy=dom_ccy,
        accrual_basis=accrual_basis,
        purchase_price=purchase_price,
        repo_rate=repo_rate,
        collateral=collateral,
    )
    
    repo_definition = lm.InstrumentDefinition(
        name=name,
        identifiers={"ClientInternal": lm.InstrumentIdValue(identifier)},
        definition=repo_instrument
    )
    
    upsert_request = {identifier: repo_definition}
    upsert_response = instruments_api.upsert_instruments(request_body=upsert_request)
    repo_luid = upsert_response.values[identifier]
    print(repo_luid)

In [7]:
# Create collateral instrument
collateral_instrument = create_collateral_instrument_definition(
    datetime(2015, 1, 1, 00, tzinfo=pytz.utc),
    datetime(2045, 1, 1, 00, tzinfo=pytz.utc),
    "GBP",
    1.01,
    0.03,
    "GBP",
    "6M",
    "F",
    "Actual360",
    [],
    [],
    0,
    0
)
print(collateral_instrument)

{'calculation_type': 'Standard',
 'coupon_rate': 0.03,
 'dom_ccy': 'GBP',
 'ex_dividend_days': None,
 'first_coupon_pay_date': None,
 'flow_conventions': {'accrual_date_adjustment': None,
                      'code': None,
                      'currency': 'GBP',
                      'day_count_convention': 'Actual360',
                      'leap_days_included': None,
                      'payment_calendars': [],
                      'payment_frequency': '6M',
                      'reset_calendars': [],
                      'reset_days': 0,
                      'roll_convention': 'F',
                      'scope': None,
                      'settle_days': 0},
 'identifiers': {},
 'initial_coupon_date': None,
 'instrument_type': 'Bond',
 'maturity_date': datetime.datetime(2045, 1, 1, 0, 0, tzinfo=<UTC>),
 'principal': 1.01,
 'start_date': datetime.datetime(2015, 1, 1, 0, 0, tzinfo=<UTC>)}


In [8]:
repo_luid_a = create_repo_instrument(
    "Repo Example Instrument A",
    "REPO_EXAMPLE_A",
    datetime(2020, 1, 1, 00, tzinfo=pytz.utc),
    datetime(2030, 1, 1, 00, tzinfo=pytz.utc),
    "USD",
    "Actual360",
    2000.00,
    1.03,
    []
)

{'asset_class': 'InterestRates',
 'dom_ccy': 'USD',
 'href': 'https://fbn-ryan-g.lusid.com/api/api/instruments/LusidInstrumentId/LUID_00003JLC?scope=default',
 'identifiers': {'ClientInternal': 'REPO_EXAMPLE_A',
                 'LusidInstrumentId': 'LUID_00003JLC'},
 'instrument_definition': {'accrual_basis': 'Actual360',
                           'collateral': [],
                           'collateral_value': None,
                           'dom_ccy': 'USD',
                           'haircut': None,
                           'instrument_type': 'Repo',
                           'margin': None,
                           'maturity_date': datetime.datetime(2030, 1, 1, 0, 0, tzinfo=tzlocal()),
                           'purchase_price': 2000.0,
                           'repo_rate': 1.03,
                           'repurchase_price': None,
                           'start_date': datetime.datetime(2020, 1, 1, 0, 0, tzinfo=tzlocal())},
 'links': None,
 'lookthrough_portfolio': N

In [9]:
repo_luid_b = create_repo_instrument(
    "Repo Example Instrument B",
    "REPO_EXAMPLE_B",
    datetime(2025, 1, 1, 00, tzinfo=pytz.utc),
    datetime(2035, 1, 1, 00, tzinfo=pytz.utc),
    "USD",
    "Actual360",
    1000.00,
    1.02,
    [collateral_instrument]
)

{'asset_class': 'InterestRates',
 'dom_ccy': 'USD',
 'href': 'https://fbn-ryan-g.lusid.com/api/api/instruments/LusidInstrumentId/LUID_00003JLD?scope=default',
 'identifiers': {'ClientInternal': 'REPO_EXAMPLE_B',
                 'LusidInstrumentId': 'LUID_00003JLD'},
 'instrument_definition': {'accrual_basis': 'Actual360',
                           'collateral': [{'calculation_type': 'Standard',
                                           'coupon_rate': 0.03,
                                           'dom_ccy': 'GBP',
                                           'ex_dividend_days': None,
                                           'first_coupon_pay_date': None,
                                           'flow_conventions': {'accrual_date_adjustment': 'Adjusted',
                                                                'code': None,
                                                                'currency': 'GBP',
                                                                'day

## Loading Repo Transactions in the Portfolio

We will continue by loading in the two repos using the transactions below.

In [10]:
transactions_df = pd.read_csv("repo_transactions.csv")
transactions_df.head()

,txn_id,type,client_id,trade_date,settlement_date,quantity,price,total_consideration,currency,portfolio
0,txn001,StockIn,REPO_EXAMPLE_A,2020-01-01T00:00:00Z,2020-01-03T00:00:00Z,75,200,"15,000.00",USD,repoExamplePortfolio
1,txn002,StockIn,REPO_EXAMPLE_B,2020-01-01T00:00:00Z,2020-01-03T00:00:00Z,100,220,"22,000.00",USD,repoExamplePortfolio


In [11]:
for portfolio_code, grouped_df in transactions_df.groupby("portfolio"):

    transaction_request = [
        lm.TransactionRequest(
            transaction_id=row["txn_id"],
            type=row["type"],
            instrument_identifiers={
                "Instrument/default/ClientInternal": row["client_id"]
            },
            transaction_date=to_date(row["trade_date"]).isoformat(),
            settlement_date=to_date(row["settlement_date"]).isoformat(),
            units=row["quantity"],
            transaction_price=lm.TransactionPrice(price=row["price"], type="Price"),
            total_consideration=lm.CurrencyAndAmount(
                amount=row["total_consideration"], currency=row["currency"]
            ),
        )
        for index, row in grouped_df.iterrows()
    ]

    response = transaction_portfolios_api.upsert_transactions(
        scope=scope, code=portfolio_code, transaction_request=transaction_request
    )

    print(f"Transactions succesfully updated at time: {response.version.as_at_date}")

Transactions succesfully updated at time: 2023-04-18 10:11:29.040128+00:00


## Updating the Principal and the Repo Rate

Now that the repos are loaded into the portfolio, we will update the repo rate of Repo A and reduce the principal of the collateral bond in Repo B.

In [13]:
# how does this work

updated_repo_a = create_repo_instrument(
    "Repo Example Instrument A",
    "REPO_EXAMPLE_A",
    datetime(2020, 1, 1, 00, tzinfo=pytz.utc),
    datetime(2030, 1, 1, 00, tzinfo=pytz.utc),
    "USD",
    "Actual360",
    2000.00,
    1.03,
    []
)

{'asset_class': 'InterestRates',
 'dom_ccy': 'USD',
 'href': 'https://fbn-ryan-g.lusid.com/api/api/instruments/LusidInstrumentId/LUID_00003JLC?scope=default',
 'identifiers': {'ClientInternal': 'REPO_EXAMPLE_A',
                 'LusidInstrumentId': 'LUID_00003JLC'},
 'instrument_definition': {'accrual_basis': 'Actual360',
                           'collateral': [],
                           'collateral_value': None,
                           'dom_ccy': 'USD',
                           'haircut': None,
                           'instrument_type': 'Repo',
                           'margin': None,
                           'maturity_date': datetime.datetime(2030, 1, 1, 0, 0, tzinfo=tzlocal()),
                           'purchase_price': 2000.0,
                           'repo_rate': 1.03,
                           'repurchase_price': None,
                           'start_date': datetime.datetime(2020, 1, 1, 0, 0, tzinfo=tzlocal())},
 'links': None,
 'lookthrough_portfolio': N